## Recruit Restaurant Visitor Prediction

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

PATH='data/'

In [4]:
table_names = ['air_reserve', 'air_store_info', 'air_visit_data', 'date_info',
               'hpg_reserve', 'hpg_store_info', 'store_id_relation']

We'll be using the popular data manipulation framework `pandas`. Among other things, pandas allows you to manipulate tables/data frames in python as one would in a database.

We're going to go ahead and load all of our csv's as dataframes into the list `tables`.

In [5]:
tables = [pd.read_csv(f'{PATH}{fname}.csv', low_memory=False) for fname in table_names]

In [6]:
from IPython.display import HTML

We can use `head()` to get a quick look at the contents of each table:
* air_reserve - reservations made in the air system. `reserve_datetime` indicates the time when the reservation was created, whereas the `visit_datetime` is when the visit will occur     
* air_store_info - info about select restaurants
* air_visit_data - historical visit data
* hpg_reserve - reservations made in hpg system
* hpg_store_info - info about select hpg restaurants
* store_id_relation - allows joining air and hpg restaurants

In [7]:
for t in tables: display(t.head())

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7


## Data Cleaning and Feature Engineering

As a structured data problem, we necessarily have to go through all the cleaning and feature engineering, even though we're using a neural network.

In [8]:
air_rsrv, air_store, air_visit, date_info, hpg_rsrv, hpg_store, air_hpg_rel = tables

We turn state Holidays to booleans, to make them more convenient for modeling. We can do calculations on pandas fields using notation very similar (often identical) to numpy.

In [9]:
date_info.holiday_flg = date_info.holiday_flg!=0
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,True
1,2016-01-02,Saturday,True
2,2016-01-03,Sunday,True
3,2016-01-04,Monday,False
4,2016-01-05,Tuesday,False


The following extracts particular date fields from a complete datetime for the purpose of constructing categoricals.

You should always consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities. We'll add to every table with a date field.

In [10]:
add_datepart(date_info, "calendar_date", drop=False)

In [11]:
date_info.head()

,calendar_date,day_of_week,holiday_flg,calendar_Year,calendar_Month,calendar_Week,calendar_Day,calendar_Dayofweek,calendar_Dayofyear,calendar_Is_month_end,calendar_Is_month_start,calendar_Is_quarter_end,calendar_Is_quarter_start,calendar_Is_year_end,calendar_Is_year_start,calendar_Elapsed
0,2016-01-01,Friday,True,2016,1,53,1,4,1,False,True,False,True,False,True,1451606400
1,2016-01-02,Saturday,True,2016,1,53,2,5,2,False,False,False,False,False,False,1451692800
2,2016-01-03,Sunday,True,2016,1,53,3,6,3,False,False,False,False,False,False,1451779200
3,2016-01-04,Monday,False,2016,1,1,4,0,4,False,False,False,False,False,False,1451865600
4,2016-01-05,Tuesday,False,2016,1,1,5,1,5,False,False,False,False,False,False,1451952000


### Examine how the reservations work during the test period

In [12]:
# Select observations between two datetimes
air_rsrv.tail(10)

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
92368,air_8093d0b565e9dbdf,2017-05-28 19:00:00,2017-04-02 18:00:00,7
92369,air_0867f7bebad6a649,2017-05-28 23:00:00,2017-02-01 22:00:00,48
92370,air_1033310359ceeac1,2017-05-28 23:00:00,2017-04-15 22:00:00,2
92371,air_1033310359ceeac1,2017-05-28 23:00:00,2017-04-18 17:00:00,2
92372,air_12c4fb7a423df20d,2017-05-29 12:00:00,2017-04-17 20:00:00,9
92373,air_db4b38ebe7a7ceff,2017-05-29 18:00:00,2017-04-17 19:00:00,6
92374,air_1033310359ceeac1,2017-05-30 23:00:00,2017-04-19 17:00:00,8
92375,air_877f79706adbfb06,2017-05-31 20:00:00,2017-04-12 23:00:00,3
92376,air_900d755ebd2f7bbd,2017-05-31 20:00:00,2017-04-19 16:00:00,10
92377,air_3cad29d1a23209d2,2017-05-31 21:00:00,2017-04-22 10:00:00,3


So the reservations are set through the test period

In [13]:
# Convert the dates
air_rsrv["visit_date"] = air_rsrv.visit_datetime.str.split(' ', expand=True)[0]
air_rsrv.drop("visit_datetime",1,inplace=True)
air_rsrv.drop("reserve_datetime",1,inplace=True)

hpg_rsrv["visit_date"] = hpg_rsrv.visit_datetime.str.split(' ', expand=True)[0]
hpg_rsrv.drop("visit_datetime",1,inplace=True)
hpg_rsrv.drop("reserve_datetime",1,inplace=True)
air_rsrv.head()


,air_store_id,reserve_visitors,visit_date
0,air_877f79706adbfb06,1,2016-01-01
1,air_db4b38ebe7a7ceff,3,2016-01-01
2,air_db4b38ebe7a7ceff,6,2016-01-01
3,air_877f79706adbfb06,2,2016-01-01
4,air_db80363d35f10926,5,2016-01-01


Rename the visit date to make it easier for aggregation

In [14]:
air_rsrv.rename(columns={'reserve_visitors': 'air_rsrv_visitors'}, inplace=True)
hpg_rsrv.rename(columns={'reserve_visitors': 'hpg_rsrv_visitors'}, inplace=True)

Summarize the reservations by visit_datetime

In [15]:
air_rsrv_by_date = air_rsrv.groupby(['air_store_id','visit_date'], as_index=False).agg({"air_rsrv_visitors": "sum"})


hpg_rsrv_by_date = hpg_rsrv.groupby(['hpg_store_id','visit_date'], as_index=False).agg({"hpg_rsrv_visitors": "sum"})
[air_rsrv_by_date.head(),hpg_rsrv_by_date.head()]
                                                                           
                                                                           

[           air_store_id  visit_date  air_rsrv_visitors
 0  air_00a91d42b08b08d9  2016-10-31                  2
 1  air_00a91d42b08b08d9  2016-12-05                  9
 2  air_00a91d42b08b08d9  2016-12-14                 18
 3  air_00a91d42b08b08d9  2016-12-17                  2
 4  air_00a91d42b08b08d9  2016-12-20                  4,
            hpg_store_id  visit_date  hpg_rsrv_visitors
 0  hpg_001112ef76b9802c  2016-02-26                  9
 1  hpg_001112ef76b9802c  2016-03-17                  3
 2  hpg_001112ef76b9802c  2016-03-31                  5
 3  hpg_001112ef76b9802c  2016-04-05                 13
 4  hpg_001112ef76b9802c  2016-04-18                  9]

`join_df` is a function for joining tables on specific fields. By default, we'll be doing a left outer join of `right` on the `left` argument using the given fields for each table.

Pandas does joins using the `merge` method. The `suffixes` argument describes the naming convention for duplicate fields. We've elected to leave the duplicate field names on the left untouched, and append a "\_y" to those on the right.

In [16]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

Joining Strategy

air_store with air_rsrv_by_date

air_store with air_visit

hpg_store with hpg_rsrv_by_date

date_info with air_visit


air_rsrv, air_store, air_visit, date_info, hpg_rsrv, hpg_store, air_hpg_rel


Now we can outer join all of our data into a single dataframe. Recall that in outer joins everytime a value in the joining field on the left table does not have a corresponding value on the right table, the corresponding row in the new table has Null values for all right table fields. One way to check that all records are consistent and complete is to check for Null values post-join, as we do here.

Aside: Why note just do an inner join? If you are assuming that all records are complete and match on the field you desire, an inner join will do the same thing as an outer join. However, in the event you are wrong or a mistake is made, an outer join followed by a null-check will catch it. (Comparing before/after # of rows for inner join is equivalent, but requires keeping track of before/after row #'s. Outer join is easier.)

In [17]:
air_store = join_df(air_store, air_rsrv_by_date, "air_store_id")
air_store.head()


,air_store_id,air_genre_name,air_area_name,latitude,longitude,visit_date,air_rsrv_visitors
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2016-07-01,3.0
1,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2016-07-04,10.0
2,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2016-07-06,14.0
3,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2016-07-09,11.0
4,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2016-07-10,12.0


In [18]:
len(air_store[air_store.air_genre_name.isnull()])

0

In [19]:
air_visit.head(3)

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29


In [20]:
# (df['date'] > '2002-1-1 01:00:00') & (df['date'] <= '2002-1-1 04:00:00')

In [31]:
air_3d7 = air_rsrv_by_date[air_rsrv_by_date['air_store_id'] == 'air_0f0cdeee6c9bf3d7']
air_rsrv_april = air_3d7[air_3d7['visit_date'] >  '2017-04-01' ]
air_rsrv_april.head(10)

,air_store_id,visit_date,air_rsrv_visitors
1723,air_0f0cdeee6c9bf3d7,2017-04-02,14
1724,air_0f0cdeee6c9bf3d7,2017-04-03,2
1725,air_0f0cdeee6c9bf3d7,2017-04-05,3
1726,air_0f0cdeee6c9bf3d7,2017-04-06,3
1727,air_0f0cdeee6c9bf3d7,2017-04-07,25
1728,air_0f0cdeee6c9bf3d7,2017-04-08,18
1729,air_0f0cdeee6c9bf3d7,2017-04-09,5
1730,air_0f0cdeee6c9bf3d7,2017-04-10,3
1731,air_0f0cdeee6c9bf3d7,2017-04-12,8
1732,air_0f0cdeee6c9bf3d7,2017-04-13,13


In [32]:
air_store_3d7 = air_store[air_store['air_store_id'] == 'air_0f0cdeee6c9bf3d7']
air_store_april = air_store_3d7[air_store_3d7['visit_date'] >  '2017-04-01' ]
air_store_april.head(10)

,air_store_id,air_genre_name,air_area_name,latitude,longitude,visit_date,air_rsrv_visitors
118,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2017-04-02,14.0
119,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2017-04-03,2.0
120,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2017-04-05,3.0
121,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2017-04-06,3.0
122,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2017-04-07,25.0
123,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2017-04-08,18.0
124,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2017-04-09,5.0
125,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2017-04-10,3.0
126,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2017-04-12,8.0
127,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2017-04-13,13.0


In [33]:
air_store_t = join_df(air_store, air_rsrv_by_date, ["air_store_id","visit_date"])
# visit_diff_from_rsrv = air_store_t[air_store_t.air_rsrv_visitors != air_store_t.air_rsrv_visitors_y]
# len(visit_diff_from_rsrv)
air_store_t.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude,visit_date,air_rsrv_visitors,air_rsrv_visitors_y
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2017-04-02,14.0,14
1,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2017-04-03,2.0,2
2,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2017-04-05,3.0,3
3,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2017-04-06,3.0,3
4,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,2017-04-07,25.0,25
